<a href="https://colab.research.google.com/github/effection00/Algorithm/blob/master/%EC%9D%80%ED%96%89%20%EC%9A%B0%EC%88%98%EA%B3%A0%EA%B0%9D%20%EC%98%88%EC%B8%A1/%20%EB%B6%84%EC%84%9D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [578]:
from google.colab import drive
drive.mount('/content/dirve')

Drive already mounted at /content/dirve; to attempt to forcibly remount, call drive.mount("/content/dirve", force_remount=True).


In [579]:
import pandas as pd

loan = pd.read_csv("/content/dirve/MyDrive/loan 수정.csv")
trans = pd.read_csv("/content/dirve/MyDrive/trans 수정.csv")

In [580]:
trans.head(3)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN


In [581]:
loan.head(3)

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,B
1,5316,1801,930711,165960,36,4610,A
2,6863,9188,930728,127080,60,2118,A


# disposition, client 데이터셋 Merge

In [582]:
disp = pd.read_csv("/content/dirve/MyDrive/new_disposition.csv")
client = pd.read_csv("/content/dirve/MyDrive/new_client.csv")

In [583]:
client.head(1)

,Unnamed: 0,client_id,birth_number,district_id,gender,age,age_levels
0,1,1,701213,18,FEMALE,29,ADULT


In [584]:
disp.head(1)

,Unnamed: 0,disp_id,client_id,account_id,type
0,1,1,1,1,OWNER


In [585]:
new = pd.merge(client,disp,how = 'inner',on='client_id')
# 본인명의 신용카드 고객
new = new[new['type']=='OWNER']
new = new.drop(['Unnamed: 0_x','Unnamed: 0_y','client_id','disp_id','type'],axis=1)

#transaction 데이터 EDA 및 랭글링

In [586]:
# 출금에는 현금(VYBER), 다른 은행으로 송금(PREVOD NA UCET), 신용카드 인출(VYBER KARTOU)
trans[trans['type'] == 'VYDAJ']['operation'].unique()

array(['VYBER', 'PREVOD NA UCET', 'VYBER KARTOU'], dtype=object)

In [587]:
# 입금에는 현금 입금(VKLAD), 다른 은행에서 입금(PREVOD Z UCTU), 알수 없음
trans[trans['type'] == 'PRIJEM']['operation'].unique()

array(['VKLAD', 'PREVOD Z UCTU', nan], dtype=object)

In [588]:
# 다른 은행으로 송금할 때만 bank 값 있음
trans[trans['operation']=='PREVOD Z UCTU']

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
15,637742,2177,930105,PRIJEM,PREVOD Z UCTU,5123.0,5923.0,DUCHOD,YZ,62457513.0
17,232961,793,930105,PRIJEM,PREVOD Z UCTU,3401.0,4201.0,NaN,IJ,6149286.0
21,542216,1844,930107,PRIJEM,PREVOD Z UCTU,3242.0,3742.0,NaN,ST,42988401.0
24,579374,1972,930107,PRIJEM,PREVOD Z UCTU,5298.0,5698.0,DUCHOD,UV,14132887.0
46,1049882,3592,930110,PRIJEM,PREVOD Z UCTU,6007.0,6607.0,DUCHOD,MN,73166322.0
...,...,...,...,...,...,...,...,...,...,...
1047119,551866,1879,981214,PRIJEM,PREVOD Z UCTU,3039.0,18013.6,NaN,QR,19752383.0
1047121,549190,1871,981214,PRIJEM,PREVOD Z UCTU,34552.0,59688.1,NaN,GH,39485311.0
1047122,546460,1860,981214,PRIJEM,PREVOD Z UCTU,3255.0,20916.3,NaN,MN,61854898.0
1047127,605757,2062,981214,PRIJEM,PREVOD Z UCTU,64642.0,97552.6,NaN,EF,10179949.0


In [589]:
# 결측치 확인
trans.isnull().sum()

trans_id           0
account_id         0
date               0
type               0
operation     178663
amount             0
balance            0
k_symbol      478646
bank          775067
account       753573
dtype: int64

In [590]:
# 마지막 데이터 날짜 
trans['date'].max()

981219

In [591]:
# urok일 때만 operation이 null이므로 urok으로 대체
trans['operation'] = trans['operation'].fillna('UROK')

In [ ]:
d

#랭글링

In [592]:
# 한 계좌를 대표로 조회한 결과 전체 은행을 합친 잔금 표시
# 계좌 총 입금 금액
# 계좌 총 출금 금액 
# 현재 잔금(balance)특성 만들기
# 이용 은행 수 
# 거래 특성 수 
# 입금 횟수, 출금 횟수
# trans_id, bank, account drop 
# bank : 송금한 은행 정보

In [593]:
# 잔고가 마이너스였던 횟수
List=[]
for i in trans['balance']:
  if i < 0:
   a = 1 
  else:
   a = 0 
  List.append(a)
trans['minus'] = List

minus = pd.DataFrame(trans.groupby('account_id')['minus'].sum(),columns=['minus'])

In [594]:
#총입금거래액
pr = trans[trans['type'] == 'PRIJEM']
pr = pd.DataFrame(pr.groupby('account_id')['amount'].sum())
pr.columns = ['credit amount']

In [595]:
#총출금거래액
vy = trans[trans['type'] == 'VYDAJ']
vy = pd.DataFrame(vy.groupby('account_id')['amount'].sum())
vy.columns = ['debit amount']

In [596]:
#금융거래횟수
trans_count = pd.DataFrame(trans.groupby('account_id')['trans_id'].count())
trans_count.columns = ['trans_count']

In [597]:
# 0으로 빈 값 채운 후 계산
trans['bank'] = trans['bank'].fillna(0)

In [598]:
# 현재 잔고 
balance = pd.DataFrame(trans.groupby('account_id')['balance'].last())
balance.columns = ['last_balance']

In [599]:
# k_symbol 원핫인코딩 후 각 거래특성 별 이용횟수 구하기
sym = pd.get_dummies(trans['k_symbol'])
trans = pd.concat([trans,sym],axis=1)

In [600]:
trans.head(1)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,minus,,DUCHOD,POJISTNE,SANKC. UROK,SIPO,SLUZBY,UROK,UVER
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,0,NaN,0,0,0,0,0,0,0,0,0


In [601]:
# 마이너스 잔액 이자 지불 횟수
min_inter = pd.DataFrame(trans.groupby('account_id')['SANKC. UROK'].sum())
min_inter.columns = ['min_inter']

In [602]:
# 대출 결제 횟수 
uver = pd.DataFrame(trans.groupby('account_id')['UVER'].sum())
uver.columns = ['uver']

In [603]:
# 이용할 칼럼만 빼고 나머지 drop
trans = trans.drop(['DUCHOD','SLUZBY','POJISTNE','SIPO','UROK'],axis=1)

In [604]:
operation = pd.get_dummies(trans['operation'])
trans = pd.concat([trans,operation],axis=1)
trans = trans.drop(['operation'],axis=1)

In [605]:
# 다른 은행으로 출금, 다른 은행에서 입금, 현금 입금, , 현금출금, 신용카드 출금의 거래 횟수 
type = pd.DataFrame(trans.groupby('account_id')['PREVOD NA UCET',	'PREVOD Z UCTU',	'VKLAD',	'VYBER',	'VYBER KARTOU'].sum())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [606]:
date = pd.DataFrame(trans.groupby('account_id')['date'].apply(sorted))

# 리스트의 [0]는 첫 이용일, 리스트의 [-1]은 마지막 이용일
first =[]
end = []
for i in date['date']:
  a = i[0]
  b = i[-1]
  first.append(a)
  end.append(b)

In [607]:
# 새로운 date 데이터프레임 형성
first_date = pd.DataFrame(first,columns=['first_date'], index=date.index)
end_date = pd.DataFrame(end,columns=['end_date'], index=date.index)
date = first_date.join(end_date)

date['end_date'] = pd.to_datetime(date['end_date'].astype(str),format='%y%m%d')
date['first_date'] = pd.to_datetime(date['first_date'].astype(str),format='%y%m%d')
date['period'] = date['end_date']-date['first_date']
date['period'] = date['period'].dt.days

date['today'] = pd.to_datetime(981219,format='%y%m%d')
date['unused'] = (date['today'] - date['end_date']).dt.days

#  이용기간, 이용하지 않은 기간 추출
date =date.drop(['first_date','end_date','today'],axis=1)
date.head(1)

,period,unused
account_id,,
1,1360,6


In [608]:
wrang = pr.join([vy,minus,trans_count,balance,min_inter,uver,type,date])

In [609]:
# 거래한 은행 수 
bank = trans.groupby('account_id')['bank'].unique()
List=[]
for i in bank:
  if 0 in i:
    a = len(i) - 1
  else:
    a = len(i)
  List.append(a)
wrang['use_bank'] = List

In [610]:
#new index를 account_id로 변경 : join위해
new.index = new['account_id']
new = new.drop(['account_id'], axis=1)
new2 = new.join([wrang])

In [611]:
new2.head()

,birth_number,district_id,gender,age,age_levels,credit amount,debit amount,minus,trans_count,last_balance,min_inter,uver,PREVOD NA UCET,PREVOD Z UCTU,VKLAD,VYBER,VYBER KARTOU,period,unused,use_bank
account_id,,,,,,,,,,,,,,,,,,,,
1,701213,18,FEMALE,29,ADULT,194267.9,180854.0,0,238,13413.9,0,0,40.0,45.0,12.0,97.0,0.0,1360,6,2
2,450204,1,MALE,54,MIDDLE AGED,1596896.8,1336983.8,0,477,42471.3,0,24,89.0,70.0,12.0,237.0,0.0,2120,2,2
3,561201,5,MALE,43,MIDDLE AGED,172849.4,119402.8,0,115,53446.5,0,0,39.0,0.0,28.0,31.0,0.0,522,8,3
4,190922,12,MALE,80,SENIOR,192210.3,158625.4,0,185,33585.0,0,0,59.0,34.0,1.0,62.0,0.0,1026,6,1
5,290125,15,MALE,70,SENIOR,97362.9,68446.4,0,81,28916.2,0,0,14.0,19.0,1.0,33.0,0.0,567,1,2


In [612]:
loan = loan[['account_id','status']]

In [613]:
final = pd.merge(new2,loan,how = 'outer',on = 'account_id')
final.index = final['account_id']
final = final.drop(['account_id','birth_number'],axis=1)

In [614]:
final.head()

,district_id,gender,age,age_levels,credit amount,debit amount,minus,trans_count,last_balance,min_inter,uver,PREVOD NA UCET,PREVOD Z UCTU,VKLAD,VYBER,VYBER KARTOU,period,unused,use_bank,status
account_id,,,,,,,,,,,,,,,,,,,,
1,18,FEMALE,29,ADULT,194267.9,180854.0,0,238,13413.9,0,0,40.0,45.0,12.0,97.0,0.0,1360,6,2,NaN
2,1,MALE,54,MIDDLE AGED,1596896.8,1336983.8,0,477,42471.3,0,24,89.0,70.0,12.0,237.0,0.0,2120,2,2,A
3,5,MALE,43,MIDDLE AGED,172849.4,119402.8,0,115,53446.5,0,0,39.0,0.0,28.0,31.0,0.0,522,8,3,NaN
4,12,MALE,80,SENIOR,192210.3,158625.4,0,185,33585.0,0,0,59.0,34.0,1.0,62.0,0.0,1026,6,1,NaN
5,15,MALE,70,SENIOR,97362.9,68446.4,0,81,28916.2,0,0,14.0,19.0,1.0,33.0,0.0,567,1,2,NaN


# 타겟 생성

In [615]:
# 1. 대출 거래 시 상태 A

#2. (balnce 금액 상위 25%)  

#and (잔금 minus 횟수  = 0) 

#and (SANKC. UROK횟수 =0)

#and (이용횟수 상위 25%)

#and 마지막 거래 한 달 이내 


In [616]:
final['grade'] = 0

In [617]:
365*7

2555

# TARGET : A(우수고객)

In [618]:
# 대출 거래시 상태 A -- 신용카드 고객 중 대출 거래는 일부 고객만 함
final.loc[final['status'] == 'A','grade'] = '1'

In [619]:
import numpy as np
# 상위 5퍼의 데이터 추출
def percent95(data,col):
  up = np.percentile(data[col], 0.95)
  return up


# 은행마다 신용평가 모델이 다르며 자체적으로 기준을 두고 모델을 만듬 
# 내가 적당하다고 생각하는 기준으로 정함
# 이용횟수 상위 25% & 마이너스 이자 0 & 잔금 마이너스 횟수 0 & 이용기간 2년이상 & 마지막 거래가 한달 이내 
final.loc[(final['trans_count'] >= percent95(final,'trans_count'))& (final['min_inter'] == 0) & (final['minus'] == 0)&(final['period']>2555)& (final['unused'] < 30),'grade'] = '1'

In [620]:
final['grade'].value_counts()

0    4297
1     203
Name: grade, dtype: int64

# TARGET : C(위험고객)

In [621]:
(final['last_balance']).sum()

204002828.8

In [622]:
final['last_balance'].max()

155592.5

In [623]:
 np.percentile(final['last_balance'], 1)

2261.5910000000003

In [624]:
# 대출 거래 시 상태 B -- 계약이 완료됐으나 미납 중인 고객 (위험 확률 크다고 판단)
final.loc[final['status'] == 'B','grade'] = '3'


In [625]:
# 하위 10% 데이터 추출
def percent5(data,col):
  down = np.percentile(data[col], 10)
  return down
# 잔금 하위 10% & (마이너스 잔고 이자거래 횟수 1번이상 이거나 마이너스 잔고 횟수 1번 이상)
final.loc[(final['last_balance'] < percent5(final,'last_balance'))& ((final['min_inter'] > 0) | (final['minus'] > 0))] = '3'

# TARGET :B(일반고객)

In [626]:
# 그 이외는 일반고객
final.loc[final['grade'] == 0,'grade'] = '2'

In [627]:
import seaborn as sns
target = final['grade']
target.value_counts()


2    4174
1     203
3     123
Name: grade, dtype: int64

In [628]:
!pip install category_encoders 

In [629]:
final.head()

,district_id,gender,age,age_levels,credit amount,debit amount,minus,trans_count,last_balance,min_inter,uver,PREVOD NA UCET,PREVOD Z UCTU,VKLAD,VYBER,VYBER KARTOU,period,unused,use_bank,status,grade
account_id,,,,,,,,,,,,,,,,,,,,,
1,18,FEMALE,29,ADULT,194268,180854,0,238,13413.9,0,0,40,45,12,97,0,1360,6,2,NaN,2
2,1,MALE,54,MIDDLE AGED,1.5969e+06,1.33698e+06,0,477,42471.3,0,24,89,70,12,237,0,2120,2,2,A,1
3,5,MALE,43,MIDDLE AGED,172849,119403,0,115,53446.5,0,0,39,0,28,31,0,522,8,3,NaN,2
4,12,MALE,80,SENIOR,192210,158625,0,185,33585,0,0,59,34,1,62,0,1026,6,1,NaN,2
5,15,MALE,70,SENIOR,97362.9,68446.4,0,81,28916.2,0,0,14,19,1,33,0,567,1,2,NaN,2


#데이터 나누기 : feature와 target

In [545]:
#leakage방지를 위해 grade 생성시 사용한 특성 제거(비금융적 정보로 우수고객 예측하기) 
final = final.drop([],axis=1)

In [546]:
# 은행의 거래 금액 큰 고객 가려내기 고객 등급 매기기 
target = 'grade'

#leakage방지를 위해 grade 생성시 사용한 특성 제거(비금융적 정보로 우수고객 예측하기) 

train_y = final[target]
train_X = final.drop([target,'status','last_balance','trans_count','min_inter','minus','period','unused'],axis=1)

#baseline 모델

In [547]:
train_y.value_counts()

2    4174
1     203
3     123
Name: grade, dtype: int64

In [548]:
base_y = pd.Series([2]*len(train_y))

In [549]:
train_y = train_y.astype(int)

In [550]:
from sklearn.metrics import f1_score
f1_score(train_y, base_y,average='weighted')

0.8926946942330848

In [551]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(train_X,train_y, train_size=0.8,random_state=24)

# 모델 생성

## RandomFrest 모델

In [553]:

!pip install imblearn

In [631]:
import numpy as np
from sklearn.pipeline import make_pipeline,Pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import cross_val_score

# 경고제거 코드
np.seterr(divide='ignore', invalid='ignore')


# gender 특성은 onehot , age_levels는 나이에 따른 level이라고 생각해 ordinal로 인코딩 
pipe =  Pipeline([
        ('preprocessing', make_pipeline(OneHotEncoder(cols=['gender']),OrdinalEncoder(cols=['age_levels']), RobustScaler())),
   ('rf',RandomForestClassifier(max_depth=20, max_features= 0.8009300134244958, n_estimators= 313))
])



In [554]:
# train_X에 전처리 적용
train_X2 = pipe.named_steps['preprocessing'].fit_transform(train_X)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [555]:
# 데이터 불균형으로 인한 과적합으로 언더샘플링 실행한 Train 데이터
from imblearn.over_sampling import  ADASYN
sam =  ADASYN(random_state=42)
x_sam,y_sam = sam.fit_resample(train_X2,train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


- 교차검증

In [577]:
# 교차검증 
scores = cross_val_score(pipe.named_steps['rf'], x_sam,y_sam, cv=3, 
                         scoring='f1_macro')
scores.mean()

0.9617133754983903

- 마지막 테스트셋 검사

In [560]:
pipe.named_steps['rf'].fit(x_sam,y_sam)
pred = pipe.predict(test_X)
f1_score(test_y,pred, average='weighted')

0.9636167171411267

- 하이퍼파라미터 RandomizeCV

In [571]:
from scipy.stats import randint, uniform
dists = {
    'n_estimators': randint(50, 500), 
    'max_depth': [5, 10, 15, 20, None], 
    'max_features': uniform(0, 1) 
}

In [572]:
from sklearn.model_selection import RandomizedSearchCV
clf = RandomizedSearchCV(
    pipe.named_steps['rf'], 
    param_distributions=dists, 
    n_iter=50, 
    cv=3, 
    scoring='accuracy',  
    verbose=1,
   
    n_jobs=-1

)
clf.fit(x_sam, y_sam)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  9.3min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [573]:
clf.best_params_

{'max_depth': 20, 'max_features': 0.8009300134244958, 'n_estimators': 313}

In [ ]:
#가장 성능 좋은 모델 셋팅
clf.best_estimator_

In [ ]:
# k=10인 교차검증 수행
# pipeline일 경우 model 대신 pipe로 넣기

score = cross_val_score(pipe, train_X, train_y,scoring='accuracy',cv=10, n_jobs=-1)
score.mean()

In [ ]:
from sklearn.metrics import accuracy_score

# 테스트 셋으로 마지막 확인
pred = pipe.predict(test_X)
accuracy_score(test_y, pred)

In [ ]:
# 원핫 적용으로 칼럼이 늘어남
# named_steps 두 번 적용

onehot = pipe.named_steps['preprocessing'].named_steps['onehotencoder']
col = onehot.transform(test_X).columns
mask = pipe.named_steps['preprocessing'].named_steps['selectkbest'].get_support()
# 중요 속성 20개
col[mask]

In [ ]:
# 특성 중요도(feature_importances_) 뽑기
fi = pd.DataFrame(model.feature_importances_, index=col[mask], columns=['feature_importance'])
fi =fi.sort_values(by=['feature_importance'], ascending=False)
fi

In [ ]:
!pip install eli5

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
     pipe.named_steps['rf'],
     scoring='accuracy',
     n_iter=5,
     random_state=24
)

train_X_processing  =pipe.named_steps['preprocessing'].transform(train_X)
#  특성별 스코어 계산
permuter.fit(train_X_processing,train_y)

In [ ]:
permuter

In [ ]:
# -인 특성을 제외해도 성능에 거의 영향이 없음
# featrue importance 
pd.Series(permuter.feature_importances_, col[mask]).sort_values(ascending=False)

In [ ]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names= col[mask].tolist()
)

# XGBoost 모델 + 하이퍼파라미터 randomizeCV

In [ ]:
from xgboost import XGBClassifier

# 경고제거 코드
np.seterr(divide='ignore', invalid='ignore')


pipe2 = make_pipeline(
    OneHotEncoder(),
    StandardScaler(),
    SelectKBest(f_regression, k=20),
    XGBClassifier(learning_rate = 0.07, max_depth =  8, min_child_weight=1, n_estimators= 809,subsample= 0.89))



In [ ]:
from scipy.stats import randint, uniform
dists =  {'xgbclassifier__n_estimators': randint(150, 1000),
              'xgbclassifier__learning_rate':uniform(0.01, 0.6),
              'xgbclassifier__subsample': uniform(0.3, 0.9),
              'xgbclassifier__max_depth': [3, 4, 5, 6, 7, 8, 9],
              'xgbclassifier__min_child_weight': [1, 2, 3, 4]
             }

In [ ]:
clf2 = RandomizedSearchCV(
    pipe2, 
    param_distributions =dists, 
    n_iter=50, 
    cv=3, 
    scoring='accuracy',  
    verbose=1,
       n_jobs=-1
)
clf2.fit(train_X, train_y)

In [ ]:
clf2.best_params_

In [ ]:
score = cross_val_score(pipe2, train_X, train_y,scoring='accuracy',cv=10, n_jobs=-1)
score.mean()

In [ ]:
pipe2.fit(train_X, train_y)
pred = pipe2.predict(test_X)
accuracy_score(test_y, pred)

In [ ]:
pipe.fit(train_X, train_y)

In [ ]:


# leakage 방지를 위해 일부 feature 제거

new3['']